#  **Gestión de propiedades y reservas de apartamentos en un portafolio**

## **Calendar** 

Muestra la representación detallada de la disponibilidad y el estado de las propiedades (apartamentos) en un determinado período de tiempo. Aquí está una explicación más detallada de cada campo en el bloque "Calendar":

1. __PropertyId (Id de la propiedad)__: Es una identificación única asociada a cada propiedad, que probablemente se refiere a un apartamento específico dentro del portafolio.

2. __CalendarDate (Fecha en el calendario)__: Representa una fecha específica entre el 01/07/2022 y el 31/12/2022.

3. __Blocked (Estado)__:
    * 0 (No reservado): Indica que el apartamento estaba disponible en esa fecha.
    * 1 (Bloqueado): Significa que el apartamento estaba cerrado y no estaba disponible para reservar en esa fecha.
    * 2 (Reservado): Indica que el apartamento estaba ocupado por clientes en esa fecha.

4. __Property_BookingId (Id de reserva de la propiedad)__: Este campo solo está presente si el estado (Blocked) es igual a 2, lo que significa que la propiedad estaba reservada en esa fecha. En este caso, el __Property_BookingId__ sería el identificador único asociado a esa reserva específica.

## **Propiedades (Properties)**
Proporciona información estática sobre las propiedades del portafolio, como su identificación única y la capacidad de personas que pueden alojarse en cada apartamento.

1. __PropertyId (Id de la propiedad)__: Es una identificación única asociada a cada propiedad (apartamento) en el portafolio.

2. __PR_Sleeps (Capacidad para dormir)__: Indica la capacidad del apartamento en personas, es decir, cuántas personas pueden dormir en ese apartamento.

## **Reservas (Bookings)**
Proporciona información detallada sobre las reservas realizadas, incluyendo el costo total, duración, fecha de creación y el canal a través del cual se realizó la reserva.

1. __Property_BookingId (Id de reserva de la propiedad)__: Identificación única asociada a una reserva específica.

2. __PB_TotalPrice (Precio total de la reserva)__: Representa el costo total de la reserva.

3. __PB_NumNights (Número de noches de la reserva)__: Indica cuántas noches está programada la reserva.

4. __PB_BookingCreatedDate (Fecha de creación de la reserva)__: Muestra la fecha en que se creó la reserva.

5. __PB_BookingChannelOriginId (Id del canal de origen de la reserva)__: Identificación del canal a través del cual se realizó la reserva.

In [5]:
# librerias
import pandas as pd

In [6]:
# Especifica la ruta del archivo Excel
archivo_excel = 'data/Data analysis test.xlsx'

# Carga el archivo Excel en un DataFrame de pandas
calendar    = pd.read_excel(archivo_excel, sheet_name= 'Calendar')
Properties  = pd.read_excel(archivo_excel, sheet_name= 'Properties')
Bookings    = pd.read_excel(archivo_excel, sheet_name= 'Bookings')


## Visión general de la ocupación mensual 

* Calcula la ocupación mensual para cada propiedad en el portafolio. Utiliza los datos del bloque "Calendar" para contar las noches bloqueadas y las noches reservadas en cada mes.
* La ocupación sera igual : 

         Ocupación (%) = (Noches Disponible / Noches Reservadas) * 100 = (((Fecha maxina - Fecha minima) - Noches Bloqueadad) / Noches Reservadas) * 100

In [27]:
# Suponiendo que 'CalendarDate' es una columna de tipo datetime en tu DataFrame 'calendar'
calendar['CalendarDate'] = pd.to_datetime(calendar['CalendarDate'])

# Encuentra la fecha máxima y mínima
fecha_maxima = calendar['CalendarDate'].max()
fecha_minima = calendar['CalendarDate'].min()

# Calcula la diferencia en días
diferencia_en_dias = (fecha_maxima - fecha_minima).days

print(f"Entre({fecha_maxima}) y ({fecha_minima}) hay un total de {diferencia_en_dias + 1} días")

Entre(2022-12-31 00:00:00) y (2022-07-01 00:00:00) hay un total de 184 días


In [24]:
# Convertir la columna 'CalendarDate' a objetos de fecha
calendar['CalendarDate'] = pd.to_datetime(calendar['CalendarDate'])

# Resumen según los valores únicos en 'PropertyId'
resumen = calendar.groupby('PropertyId')['Blocked'].value_counts().unstack(fill_value=0)

# Imprimir el resumen
resumen

Blocked,0,1,2
PropertyId,,,
2883,23,1,160
3963,35,1,148
3964,17,1,166
4138,21,7,156
4181,22,1,161
...,...,...,...
43368,184,0,0
43371,13,0,6
43372,1,0,0


In [31]:
# Obtener un resumen de los valores faltantes en cada columna
resumen_valores_faltantes = calendar.isnull().sum()

# Obtener el porcentaje de valores faltantes en cada columna
porcentaje_valores_faltantes = (calendar.isnull().mean()) * 100

# Crear un DataFrame con el resumen
resumen = pd.DataFrame({
    'Valores faltantes': resumen_valores_faltantes,
    'Porcentaje de valores faltantes': porcentaje_valores_faltantes
})

# Mostrar el resumen
print(resumen)

                    Valores faltantes  Porcentaje de valores faltantes
PropertyId                          0                         0.000000
CalendarDate                        0                         0.000000
Blocked                             0                         0.000000
Property_BookingId               7725                        19.862186
